In [1]:
from __future__ import print_function # Python 2/3 compatibility
import boto3

In [32]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://localhost:8000")

table = dynamodb.Table('ProductCatalog')

table.delete()

table = dynamodb.create_table(
    TableName='ProductCatalog',
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'Title',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'Title',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)

Table status: ACTIVE


In [33]:
# load data
import json
import decimal

dynamodb = boto3.resource('dynamodb', region_name='us-west-2', endpoint_url="http://localhost:8000")

table = dynamodb.Table('ProductCatalog')

with open("ProductCatalog_data.json") as json_file:
    prods = json.load(json_file, parse_float = decimal.Decimal)
    for prod in prods:
        Id = int(prod['Id'])
        Title = prod['Title']
        ISBN = prod['ISBN']
        Authors = set(prod['Authors'])
        Price = decimal.Decimal(prod['Price'])
        Dim = prod['Dimensions']
        PageCount = int(prod['PageCount'])
        InPub = prod['InPublication']
        cate = prod['ProductCategory']
        reviews = prod['reviews'];
       
        print("Adding product catalog:", Id, Title)

        response = table.put_item(
           Item={
               'Id': Id,
               'Title': Title,
               'ISBN': ISBN,
               'Authors' : Authors,
               'Price': Price,
               'Dimensions': Dim,
               'PageCount' : PageCount,
               'InPublication' : InPub,
               'ProductCategory' : cate,
               'Reviews' : reviews
            }
        )
        print("PutItem succeeded:")
        print(json.dumps(response, indent=4, cls=DecimalEncoder))

Adding product catalog: 201 Book 101 Title {'reviewEntries': []}
PutItem succeeded:
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "x-amz-crc32": "2745614147",
            "content-length": "2",
            "x-amzn-requestid": "197d2a2d-e669-4f8a-a740-c70895bc8971",
            "content-type": "application/x-amz-json-1.0",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0,
        "HTTPStatusCode": 200,
        "RequestId": "197d2a2d-e669-4f8a-a740-c70895bc8971"
    }
}
Adding product catalog: 102 Book 102 Title {'reviewEntries': []}
PutItem succeeded:
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "x-amz-crc32": "2745614147",
            "content-length": "2",
            "x-amzn-requestid": "f06b9670-53bc-48a0-a69c-3e3c7b300b95",
            "content-type": "application/x-amz-json-1.0",
            "server": "Jetty(8.1.12.v20130726)"
        },
        "RetryAttempts": 0,
        "HTTPStatusCode": 200,
        "R

In [43]:
# get single item
import boto3
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

class MyEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, set):
          return list(o)
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource("dynamodb", region_name='us-west-2', endpoint_url="http://localhost:8000")

table = dynamodb.Table('ProductCatalog')

Title = "Book 401 Title"
Id = 401

try:
    response = table.get_item(
        Key={
            'Id': Id,
            'Title': Title
        }
    )
    #print(response['Item'].Reviews.reviewEntries[0].reviewedOn)
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    item = response['Item']
    #print(item['Reviews']['reviewEntries'][0]['reviewRank'])
    print("GetItem succeeded:")
    print(json.dumps(item, indent=4, cls=MyEncoder))

FOUR
GetItem succeeded:
{
    "Reviews": {
        "reviewEntries": [
            {
                "reviewRank": "FOUR",
                "reviewedOn": "2017-02-25",
                "reviewer": "Imaya",
                "reviewText": "Very good book"
            },
            {
                "reviewRank": "ONE",
                "reviewedOn": "2016-02-24",
                "reviewer": "Chitra",
                "reviewText": "Very bad book"
            }
        ]
    },
    "InPublication": true,
    "Price": 19.99,
    "Dimensions": "8.5 x 11.0 x 0.2",
    "Title": "Book 401 Title",
    "Authors": [
        "SomeOne",
        "Something"
    ],
    "Id": 401,
    "ProductCategory": "Book",
    "PageCount": 210,
    "ISBN": "123-45678782"
}
